In [4]:
from datasets import load_dataset

dataset = load_dataset("autopilot-ai/Gujarati-Grammarly-Datasets",data_files=["sentence-pairs/gujarati_sentences_1M.csv"], split = "train[100000:110000]")

Found cached dataset csv (/root/.cache/huggingface/datasets/autopilot-ai___csv/autopilot-ai--Gujarati-Grammarly-Datasets-7432f8bd995d09c0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


In [5]:
dataset

Dataset({
    features: ['Correct', 'Incorrect'],
    num_rows: 10000
})

In [9]:
dataset[2]

{'Correct': 'ત્યાં કોઈ કારણ નથી કે શા માટે વધુ રમતો દૈનિક સ્તરોના જોડાણ-પ્રેરિત હૂકનો ઉપયોગ કરી શકતા નથી.',
 'Incorrect': 'ત્યાં કૌઇ કારણ નથી કૈ શા માતૈ વધુ રમતો દૈનિક સ્તરોના જોડાણ-પ્રેરિત હૂકનો ઉપયોગ કરી શકતા નથી.'}

In [10]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

checkpoint = "Jayveersinh-Raj/guj-grammar-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda115.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 115
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda115.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


In [11]:
model.to("cuda")

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 1024)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 1024)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=Fals

In [13]:
import torch
# Input sentence for prediction
input_sentence = "શિયાળાની ઠંડી હોય તો મોટાભાગના લોકોને શરદી, ઉધરસ અને છીંક આવવાની ટકલિફ હૌય છે."

# Tokenize the input sentence
input_ids = tokenizer.encode(input_sentence, return_tensors="pt").to("cuda")

# Generate predictions
with torch.no_grad():
    output_ids = model.generate(input_ids, max_new_tokens=200)

# Decode the generated output
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the generated output
print("Input Sentence:", input_sentence)
print("Generated Sentence:", output_sentence)

Input Sentence: શિયાળાની ઠંડી હોય તો મોટાભાગના લોકોને શરદી, ઉધરસ અને છીંક આવવાની ટકલિફ હૌય છે.
Generated Sentence: શિયાળાની ઠંડી હોય તો મોટાભાગના લોકોને શરદી, ઉધરસ અને છીંક આવવાની તર્કીલ હોય છે.


In [14]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Create lists to store input, ground truth, and generated sentences
input_sentences = []
ground_truth_sentences = []
generated_sentences = []

# Iterate through the dataset
for example in tqdm(dataset, desc="Generating predictions"):
    # Assuming 'Incorrect' is the key for incorrect sentences
    input_sentence = example['Incorrect']
    
    # Tokenize the input sentence
    input_ids = tokenizer.encode(input_sentence, return_tensors="pt").to("cuda")

    # Generate predictions
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_new_tokens=200)

    # Decode the generated output
    generated_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Append to lists
    input_sentences.append(input_sentence)
    ground_truth_sentences.append(example['Correct'])
    generated_sentences.append(generated_sentence)

# Create a DataFrame
df = pd.DataFrame({
    'Input Sentence': input_sentences,
    'Ground Truth': ground_truth_sentences,
    'Generated Sentence': generated_sentences
})

# Save the DataFrame to a CSV file
df.to_csv('predictions.csv', index=False)

# Display the DataFrame
df.head()

Generating predictions: 100%|██████████| 10000/10000 [3:59:08<00:00,  1.43s/it] 


,Input Sentence,Ground Truth,Generated Sentence
0,તે એક સુરક્ષાકર્મી હતો.,તે એક સુરક્ષાકર્મી હતો.,તે એક સુરક્ષાકર્મી હતો.
1,"મોટેભાગે, ગુલામોનું પણ વેપાર થાય છે.","મોટેભાગે, ગુલામોનું પણ વેપાર થાય છે.","મોટાભાગે, ગુલામોનું પણ વેપાર થાય છે."
2,ત્યાં કૌઇ કારણ નથી કૈ શા માતૈ વધુ રમતો દૈનિક સ...,ત્યાં કોઈ કારણ નથી કે શા માટે વધુ રમતો દૈનિક સ...,ત્યાં કોઈ કારણ નથી કે શા માટે વધુ રમતો દૈનિક સ...
3,હૂઁ વધૂ કંઇ નથી કહી શકતો કારણ કે તેમનો ટ્રેક ટ...,હું વધુ કંઇ નથી કહી શકતો કારણ કે તેમનો ટ્રેક ત...,હું વધુ કંઇ નથી કહી શકતો કારણ કે તેમનો ટ્રેક ત...
4,"ઉદાહરણ તરીકે, લખો ઝીરો ફક્ત ડેટા પર ઝૂરો લખે છ...","ઉદાહરણ તરીકે, લખો ઝીરો ફક્ત ડેટા પર ઝૂરો લખે છ...",લખો ઝીરો ફક્ત ડેટા પર ઝૂરો લખે છે અને રેન્ડમ ડ...


In [21]:
df.iloc[9]

Input Sentence        સલમાન ખાનનિ 29 વર્ષ જૂની હિરૌઇન આજે પણ એટલી જ ...
Ground Truth          સલમાન ખાનની 29 વર્ષ જૂની હીરોઈન આજે પણ એટલી જ ...
Generated Sentence    સલમાન ખાનની 29 વર્ષ જૂની હિરોઇન આજે પણ એટલી જ ...
Name: 9, dtype: object

In [17]:
!pip install numba

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 3.7 MB 851 kB/s eta 0:00:01
     |████████████████████████████████| 43.6 MB 9.8 kB/s  eta 0:00:01


In [18]:
from numba import cuda
cuda.select_device(1)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7f39ab1b2ea0 to Device at 0x7f39ab1e16a0>